In [1]:
import tensorflow as tf
from sklearn.externals import joblib
# from word2vec_fast import Word2VecFast
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Flatten, Dropout, Reshape
from keras.models import Model
import keras
import numpy as np

import sys
sys.path.append('..')
from utils.word2vec_fast import *

Using TensorFlow backend.


In [2]:
wv = Word2VecFast.load_word2vec_format(file_path='../data/chip2018/word_embedding.txt', word_shape=300)
print('word_embedding shape: ', wv.word_shape())
word_size, embedding_size = wv.word_embeddings().shape[0], wv.word_embeddings().shape[1]

word_embedding shape:  (9648, 300)


In [3]:
# 加载数据集
dataset = joblib.load(filename='../data/chip2018/chip2018.data')
X_train, Y_train, X_test, Y_test = dataset['X_train'], dataset['Y_train'], dataset['X_test'], dataset['Y_test']
print('X_train shape:', X_train.shape)
Y_train = keras.utils.to_categorical(Y_train, 2)
Y_test = keras.utils.to_categorical(Y_test, 2)

X_train shape: (18000, 90)


In [4]:
sequence_length = 45
num_classes = 2

In [11]:
wv_initializer = keras.initializers.Constant(wv.word_embeddings())

input_x1 = Input(shape=(sequence_length*2,))

e1 = Embedding(output_dim=embedding_size, input_dim=word_size, embeddings_initializer=wv_initializer, input_length=sequence_length*2)(input_x1)
r1 = Reshape(target_shape=(sequence_length*2,embedding_size,1))(e1)
c1 = Conv2D(filters=128, kernel_size=(3, 300), activation='relu')(r1)
m1 = MaxPooling2D(pool_size=(88, 1), padding='VALID')(c1)
f1 = Flatten()(m1)
d1 = Dropout(rate=0.5)(f1)
out = Dense(2, activation='sigmoid')(d1)
print(m1)
# keras.utils.plot_model(model, to_file='model1.png')

Tensor("max_pooling2d_2/MaxPool:0", shape=(?, 1, 1, 128), dtype=float32)


In [12]:
model = Model(inputs=input_x1, outputs=out)
model.summary()
keras.utils.plot_model(model, to_file='model1.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 90)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 90, 300)           2894400   
_________________________________________________________________
reshape_2 (Reshape)          (None, 90, 300, 1)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 88, 1, 128)        115328    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [13]:
# 批量梯度下降
model.compile(optimizer=keras.optimizers.Adam(lr=0.005), loss='binary_crossentropy', metrics=['acc'])

In [14]:
history = model.fit(x=X_train, y=Y_train, batch_size=32, epochs=5)

Epoch 1/5
 9984/18000 [===============>..............] - ETA: 16s - loss: 0.7015 - acc: 0.4948

KeyboardInterrupt: 